In [2]:
import google.generativeai as genai
import getpass # Import the getpass library

# --- CONFIGURATION ---
GCP_PROJECT_ID = "kaggle-bigquery-dataset"

# Ask the user to input their API key securely
try:
  GEMINI_API_KEY = getpass.getpass('Please enter your Google AI Studio API Key: ')
  genai.configure(api_key=GEMINI_API_KEY)
except Exception as e:
  print("Could not configure the API key. Please make sure it is correct.")

Please enter your Google AI Studio API Key: ··········


In [3]:
!pip install pillow
from PIL import Image
import requests
from io import BytesIO

In [4]:
# Install the necessary libraries
!pip install google-cloud-bigquery google-generativeai pandas

# Authenticate your user account to access your GCP project
from google.colab import auth
auth.authenticate_user()

print("Authentication and installation complete!")

Authentication and installation complete!


In [5]:
import pandas as pd
import google.generativeai as genai
from google.colab import userdata
from google.cloud import bigquery

# --- CONFIGURATION ---
GCP_PROJECT_ID = "kaggle-bigquery-challenge"  # <-- IMPORTANT: Your GCP Project ID
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
# -------------------

# Configure the Gemini API client
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-2.5-flash-lite-preview-06-17")

# Configure the BigQuery client
bq_client = bigquery.Client(project=GCP_PROJECT_ID)

print("Clients configured successfully.")
for m in genai.list_models():
    print(m.name, "-", m.display_name)


Clients configured successfully.
models/embedding-gecko-001 - Embedding Gecko
models/gemini-1.5-pro-latest - Gemini 1.5 Pro Latest
models/gemini-1.5-pro-002 - Gemini 1.5 Pro 002
models/gemini-1.5-pro - Gemini 1.5 Pro
models/gemini-1.5-flash-latest - Gemini 1.5 Flash Latest
models/gemini-1.5-flash - Gemini 1.5 Flash
models/gemini-1.5-flash-002 - Gemini 1.5 Flash 002
models/gemini-1.5-flash-8b - Gemini 1.5 Flash-8B
models/gemini-1.5-flash-8b-001 - Gemini 1.5 Flash-8B 001
models/gemini-1.5-flash-8b-latest - Gemini 1.5 Flash-8B Latest
models/gemini-2.5-pro-preview-03-25 - Gemini 2.5 Pro Preview 03-25
models/gemini-2.5-flash-preview-05-20 - Gemini 2.5 Flash Preview 05-20
models/gemini-2.5-flash - Gemini 2.5 Flash
models/gemini-2.5-flash-lite-preview-06-17 - Gemini 2.5 Flash-Lite Preview 06-17
models/gemini-2.5-pro-preview-05-06 - Gemini 2.5 Pro Preview 05-06
models/gemini-2.5-pro-preview-06-05 - Gemini 2.5 Pro Preview
models/gemini-2.5-pro - Gemini 2.5 Pro
models/gemini-2.0-flash-exp - Gemi

In [6]:
def get_customer_history(customer_id):
  """Fetches a customer's purchase history from BigQuery."""

  # The only change is adding single quotes around {customer_id}
  sql_query = f"""
      SELECT
        purchase_history
      FROM
        `kaggle-bigquery-challenge.Sales_transaction.vw_customer_purchase_history`
      WHERE
        CustomerNo = '{customer_id}'
  """
  print(f"Running query for customer {customer_id}...")
  df = bq_client.query(sql_query).to_dataframe()

  if df.empty:
    return "No purchase history found for this customer."

  return df['purchase_history'][0]

In [7]:
def generate_customer_segment(history):
  """Uses Gemini to classify a customer based on their purchase history."""
  if "No purchase history" in history:
    return "Cannot generate segment without history."

  prompt = f"""
      Analyze this customer purchase history and classify them into ONE of the following segments
      based on the products they buy: [Holiday Shopper, Home Decorator, Gift Giver, Party Planner, Arts & Crafts Enthusiast].
      Provide only the segment name.

      History: {history}
  """
  print("Generating segment with Gemini...")
  response = model.generate_content(prompt)
  return response.text

In [8]:
def generate_marketing_slogan(segment):
  """Uses Gemini to generate a marketing slogan for a customer segment."""
  prompt = f"""
      You are a creative marketing assistant for a large retailer like WalMart.
      Based on the customer segment '{segment}', write one short, catchy SMS marketing slogan.
  """
  print(f"Generating slogan for {segment}...")
  response = model.generate_content(prompt)
  return response.text

In [9]:
# --- Let's test our FULL agent ---
customer_to_analyze = 13069

# 1. The agent fetches data from BigQuery
history = get_customer_history(customer_to_analyze)

# 2. The agent identifies the customer segment
segment = generate_customer_segment(history).strip()

# 3. The agent generates a creative marketing idea
slogan = generate_marketing_slogan(segment).strip()

# 4. The agent presents the complete result
print("\n--- 🤖 MARKETING CO-PILOT RESULT ---")
print(f"Customer ID: \t\t{customer_to_analyze}")
print(f"Identified Segment: \t{segment}")
print(f"Suggested SMS Slogan: \t'{slogan}'")

Running query for customer 13069...
Generating segment with Gemini...
Generating slogan for Holiday Shopper...

--- 🤖 MARKETING CO-PILOT RESULT ---
Customer ID: 		13069
Identified Segment: 	Holiday Shopper
Suggested SMS Slogan: 	'Here are a few options for a short, catchy SMS slogan for the 'Holiday Shopper' segment, playing on different angles:

**Focusing on Convenience/Stress Relief:**

*   **Holiday shopping? We've got it all! 🎁**

**Focusing on Value/Savings:**

*   ** Unwrap savings this holiday! 🎄 Shop now!**

**Focusing on the Festive Spirit:**

*   **Get merry & bright with holiday deals! ✨**

**Focusing on Urgency/Last Minute:**

*   **Don't miss out! Holiday magic awaits. 🎅**

I'd lean towards **"Holiday shopping? We've got it all! 🎁"** as it's inclusive, addresses a common pain point, and uses a relevant emoji.'


#At the heart of the solution lies a seamless pipeline:

The system takes an input UserID.

* This is passed into a SQL function that connects directly to BigQuery, ensuring fast and scalable access to user-specific data.

* The retrieved information is then intelligently fed into Gemini, Google’s state-of-the-art generative AI model.

* Finally, Gemini synthesizes the insights and produces a clear, contextualized response — which is then presented to the end user.

This integration creates a smooth flow from data → intelligence → actionable answer, transforming raw queries into meaningful outcome

##New Agentic Workflow:

* A user gives a natural language prompt (e.g., "show me what customer 17490 bought").

* Gemini analyzes the prompt itself and generates the SQL query needed to answer it.

* The code executes the AI-generated query against BigQuery.

* The results are displayed or passed to another AI call for summarization

In [10]:
# A more generic function to run ANY valid SQL query
def execute_bq_query(sql_query):
  """Executes a SQL query against BigQuery and returns a DataFrame."""
  print("Executing AI-generated SQL...")
  try:
    df = bq_client.query(sql_query).to_dataframe()
    return df
  except Exception as e:
    print(f"Error executing SQL: {e}")
    return None

# The new core function that turns natural language into SQL
def generate_sql_from_prompt(user_prompt):
    """Uses Gemini to convert a user's question into a BigQuery SQL query."""

    # This is the "meta-prompt" that teaches the AI.
    # We give it the rules, the table schema, and the user's question.
    system_prompt = f"""
    You are a GoogleSQL expert. Your task is to write a BigQuery SQL query based on a user's request.
    You must follow these rules:
    0. THE TABLE NAME IS CASE-SENSITIVE
    1. The ONLY tables you can query is `
        'kaggle-bigquery-challenge.Sales_transaction.vw_customer_purchase_history',
        'kaggle-bigquery-challenge.Sales_transaction.online_transaction',
        'kaggle-bigquery-challenge.Sales_transaction.product_review'.
    2. The First table has two columns: `CustomerNo` (STRING) and `purchase_history` (STRING)
       The Second table has eight columns: TransactionNo (STRING), Date (DATE), ProductNo (STRING), ProductName (STRING), Price (FLOAT), Quantity (INTEGER), CustomerNo (STRING), and Country (STRING).
       The Third Table has two column: 'ProductNo' (STRING) and 'Review' (STRING)
    3. You must wrap any string values in the WHERE clause in single quotes (e.g., `WHERE CustomerNo = '12345'`).
    4. Your response must be ONLY the SQL query, with no explanation or other text.

    User's request: "{user_prompt}"

    SQL Query:
    """

    print("Generating SQL from user prompt...")
    response = model.generate_content(system_prompt)

    # Clean up the response to get only the SQL
    sql = response.text.strip()
    if sql.startswith("```sql"):
        sql = sql[6:]
    if sql.endswith("```"):
        sql = sql[:-3]

    return sql.strip()

In [11]:
# --- Let's test our new NL-to-SQL agent ---
#user_question = "What did customer number 17490 buy?" # <-- Ask different questions here!
user_question = "List the distinct 100 productname " # <-- Try this one too

# 1. The agent converts your question into a SQL query
generated_sql = generate_sql_from_prompt(user_question)
print(f"🤖 Generated SQL: {generated_sql}")

# 2. The agent executes its own query against BigQuery
query_results = execute_bq_query(generated_sql)

# 3. The agent presents the final result
print("\n--- 🤖 AGENT QUERY RESULT ---")
if query_results is not None:
  # Pandas .to_string() prints the result in a nice table format
  print(query_results.to_string())
else:
  print("The query failed to execute.")

Generating SQL from user prompt...
🤖 Generated SQL: SELECT
    DISTINCT ProductName
FROM
    `kaggle-bigquery-challenge.Sales_transaction.online_transaction`
LIMIT 100
Executing AI-generated SQL...

--- 🤖 AGENT QUERY RESULT ---
                            ProductName
0         Set Of 2 Wooden Market Crates
1   Christmas Star Wish List Chalkboard
2              Storage Tin Vintage Leaf
3     Tree T-Light Holder Willie Winkie
4     Set Of 4 Knick Knack Tins Poppies
5               Bag 500g Swirly Marbles
6              Joy Wooden Block Letters
7            Peace Wooden Block Letters
8           T-Light Holder Hanging Lace
9             T-Light Holder White Lace
10                    Toy Tidy Spaceboy
11       Grow Your Own Flowers Set Of 3
12           Toy Tidy Dolly Girl Design
13        Set Of 3 Cake Tins Sketchbook
14        Set Of 6 Herb Tins Sketchbook
15  Squarecushion Cover Pink Union Jack
16              36 Foil Star Cake Cases
17          Triple Wire Hook Pink Heart
18          

#Phase 3: Giving Eyes to Our Model
In this stage, we elevate our system by integrating visual perception. Leveraging image datasets stored in BigQuery, we empower the model with Gemini’s advanced image understanding and visualization capabilities. This allows our solution to not just process data, but to truly see, interpret, and analyze images for deeper insights and enhanced decision-making.

In [12]:
# A more generic function to run ANY valid SQL query
def execute_bq_query(sql_query):
  """Executes a SQL query against BigQuery and returns a DataFrame."""
  print("Executing AI-generated SQL...")
  try:
    df = bq_client.query(sql_query).to_dataframe()
    return df
  except Exception as e:
    print(f"Error executing SQL: {e}")
    return None

# The new core function that turns natural language into SQL
def generate_sql_from_prompt(user_prompt):
    """Uses Gemini to convert a user's question into a BigQuery SQL query."""

    system_prompt = f"""
    You are a GoogleSQL expert. Your task is to write a single, valid BigQuery SQL query based on a user's request.
    You must follow these rules:
    0. THE TABLE NAME IS CASE-SENSITIVE
    1. You have access to the following tables:
      - `kaggle-bigquery-challenge.Sales_transaction.online_transaction` (alias t) has columns: TransactionNo(STRING), Date(DATE), ProductNo(STRING), ProductName(STRING), Price(FLOAT), Quantity(INTEGER), CustomerNo(STRING), Country(STRING).
      - `kaggle-bigquery-challenge.Sales_transaction.product_review` (alias r) has columns: ProductNo(STRING), Review(STRING).
      - `kaggle-bigquery-challenge.Sales_transaction.product_image` (alias i) has columns: ProductName(STRING), ImageUrl(STRING).
    2. You must write a single query. Do not write multiple queries.
    3. Use `SELECT DISTINCT` to avoid duplicate rows in the output.
    4. You must correctly JOIN tables using the `ProductNo` column when a user's question requires information from multiple tables.
    5. You must wrap any string values in the WHERE clause in single quotes (e.g., `WHERE t.ProductName = 'RED RETROSPOT MUG'`).
    6. Your response must be ONLY the SQL query, with no explanation, comments, or markdown.

    User's request: "{user_prompt}"

    SQL Query:
    """

    print("Generating SQL from user prompt...")
    response = model.generate_content(system_prompt)

    # Clean up the response to get only the SQL
    sql = response.text.strip().replace("```sql", "").replace("```", "")
    return sql

    # Clean up the response to get only the SQL
    sql = response.text.strip()
    if sql.startswith("```sql"):
        sql = sql[6:]
    if sql.endswith("```"):
        sql = sql[:-3]

    return sql.strip()

In [13]:
def analyze_image_from_url(image_url):
    """Takes an image URL and uses Gemini Vision to describe it."""
    try:
        response = requests.get(image_url)
        # Make sure the request was successful
        response.raise_for_status()

        img = Image.open(BytesIO(response.content))

        # --- THIS IS THE UPDATED LINE ---
        # We are now using a powerful, modern model from your list
        vision_model = genai.GenerativeModel("gemini-2.5-pro")

        print("AI is analyzing the product image...")
        # The prompt for the vision model can be simple
        response = vision_model.generate_content(["Describe this product image from a marketing perspective.", img])
        return f"\n--- Image Analysis ---\n{response.text}"
    except Exception as e:
        return f"\n--- Image Analysis ---\nCould not analyze image. Error: {e}"

In [14]:
# --- Let's test our MASTER AGENT ---
user_question = "Show me the reviews and the image for the 'Tree T-Light Holder Willie Winkie'"

# 1. The agent converts your question into a complex SQL query
generated_sql = generate_sql_from_prompt(user_question)
print(f"🤖 Generated SQL:\n{generated_sql}")

# 2. The agent executes its own query against BigQuery
results_df = execute_bq_query(generated_sql)

# 3. The agent presents the structured data results
print("\n--- 🤖 AGENT QUERY RESULT ---")
if results_df is not None:
  print(results_df.to_string())

  # 4. AGENTIC STEP: The agent inspects the results and decides to use its vision tool
  if 'ImageUrl' in results_df.columns and not results_df['ImageUrl'].empty:
      image_url = results_df['ImageUrl'][0]
      if image_url:
          # If an image URL was found, call the vision tool
          image_analysis_result = analyze_image_from_url(image_url)
          print(image_analysis_result)
else:
  print("The query failed to execute.")

Generating SQL from user prompt...
🤖 Generated SQL:
SELECT DISTINCT r.Review, i.ImageUrl FROM `kaggle-bigquery-challenge.Sales_transaction.product_review` AS r JOIN `kaggle-bigquery-challenge.Sales_transaction.online_transaction` AS t ON r.ProductNo = t.ProductNo JOIN `kaggle-bigquery-challenge.Sales_transaction.product_image` AS i ON t.ProductName = i.ProductName WHERE t.ProductName = 'Tree T-Light Holder Willie Winkie'
Executing AI-generated SQL...

--- 🤖 AGENT QUERY RESULT ---
                                Review                                                        ImageUrl
0  Decent purchase, does the job well.  https://m.media-amazon.com/images/I/71DoqS28lqL._AC_UL320_.jpg
AI is analyzing the product image...

--- Image Analysis ---
Of course! Here is a marketing description of the product image, broken down into different components.

### Overall Marketing Angle

The image showcases a **bulk value pack** of battery-operated fairy lights, targeting DIY enthusiasts, event plann

# The Hybrid Architecture , Leveraging the ML. Function

In [15]:
# A more generic function to run ANY valid SQL query
def execute_bq_query(sql_query):
  """Executes a SQL query against BigQuery and returns a DataFrame."""
  print("Executing AI-generated SQL...")
  try:
    df = bq_client.query(sql_query).to_dataframe()
    return df
  except Exception as e:
    print(f"Error executing SQL: {e}")
    return None

# The new core function that turns natural language into SQL
def generate_sql_from_prompt(user_prompt):
    """Uses Gemini to convert a user's question into a BigQuery SQL query."""

    system_prompt = f"""
    You are a GoogleSQL expert. Your task is to write a single, valid BigQuery SQL query based on a user's request.
    You must follow these rules:
    0. THE TABLE NAME IS CASE-SENSITIVE
    1. You have access to the following tables:
      - `kaggle-bigquery-challenge.Sales_transaction.online_transaction` (alias t) has columns: TransactionNo(STRING), Date(DATE), ProductNo(STRING), ProductName(STRING), Price(FLOAT), Quantity(INTEGER), CustomerNo(STRING), Country(STRING).
      - `kaggle-bigquery-challenge.Sales_transaction.product_review` (alias r) has columns: ProductNo(STRING), Review(STRING).
      - `kaggle-bigquery-challenge.Sales_transaction.product_image` (alias i) has columns: ProductName(STRING), ImageUrl(STRING).
    2. You must write a single query. Do not write multiple queries.
    3. Use `SELECT DISTINCT` to avoid duplicate rows in the output.
    4. You must correctly JOIN tables using the `ProductNo` column when a user's question requires information from multiple tables.
    5. You must wrap any string values in the WHERE clause in single quotes (e.g., `WHERE t.ProductName = 'RED RETROSPOT MUG'`).
    6 Your response must be ONLY the SQL query, with no explanation, comments, or markdown.

    **IMPORTANT RULE:** To use  model with `ML.PREDICT`, the input data MUST have the exact same features and transformations used during training.
    The model was trained on `CustomerNo` and a column named `features` which was created with the transformation `ml.TF_IDF(SPLIT(purchase_history, ', ')) OVER()`.

    Therefore, to find the cluster for customer '12345', the query MUST follow this exact structure:
    ```sql
    SELECT
      CENTROID_ID
    FROM
      ML.PREDICT(MODEL `kaggle-bigquery-challenge.Sales_transaction.customer_segment_model`,
        (
          SELECT
            CustomerNo,
            ml.TF_IDF(SPLIT(purchase_history, ', ')) OVER() AS features
          FROM
            `kaggle-bigquery-challenge.Sales_transaction.vw_customer_purchase_history`
          WHERE CustomerNo = '12345'
        )
      )
    ```
    User's request: "{user_prompt}"

    SQL Query:
    """

    print("Generating SQL from user prompt...")
    response = model.generate_content(system_prompt)

    # Clean up the response to get only the SQL
    sql = response.text.strip().replace("```sql", "").replace("```", "")
    return sql

    # Clean up the response to get only the SQL
    sql = response.text.strip()
    if sql.startswith("```sql"):
        sql = sql[6:]
    if sql.endswith("```"):
        sql = sql[:-3]

    return sql.strip()

In [16]:
# --- Let's test our MASTER AGENT ---
user_question = "Which cluster does customer 17490 belong to?"

# 1. The agent converts your question into a complex SQL query
generated_sql = generate_sql_from_prompt(user_question)
print(f"🤖 Generated SQL:\n{generated_sql}")

# 2. The agent executes its own query against BigQuery
results_df = execute_bq_query(generated_sql)

# 3. The agent presents the structured data results
print("\n--- 🤖 AGENT QUERY RESULT ---")
if results_df is not None:
  print(results_df.to_string())

  # 4. AGENTIC STEP: The agent inspects the results and decides to use its vision tool
  if 'ImageUrl' in results_df.columns and not results_df['ImageUrl'].empty:
      image_url = results_df['ImageUrl'][0]
      if image_url:
          # If an image URL was found, call the vision tool
          image_analysis_result = analyze_image_from_url(image_url)
          print(image_analysis_result)
else:
  print("The query failed to execute.")

Generating SQL from user prompt...
🤖 Generated SQL:

SELECT
  CENTROID_ID
FROM
  ML.PREDICT(MODEL `kaggle-bigquery-challenge.Sales_transaction.customer_segment_model`,
    (
      SELECT
        CustomerNo,
        ml.TF_IDF(SPLIT(purchase_history, ', ')) OVER() AS features
      FROM
        `kaggle-bigquery-challenge.Sales_transaction.vw_customer_purchase_history`
      WHERE CustomerNo = '17490'
    )
  )

Executing AI-generated SQL...

--- 🤖 AGENT QUERY RESULT ---
   CENTROID_ID
0            5
